# Final Exam &mdash; Problem 8

### SA421 Fall 2015

**You will need to run the cell below for the basic constructs.**

In [4]:
import queue
import collections
import random
import math


# An event handler.
class EventHandler:
    def __init__(self):
        self.time_now = 0
        self.event_queue = queue.PriorityQueue()

    def run(self, run_until_time=float('inf')):
        while self.time_now <= run_until_time and (not self.event_queue.empty()):
            # Get the next event from the queue
            the_event = self.event_queue.get()
            if the_event.time_occurred > run_until_time:
                break
                
            self.time_now = the_event.time_occurred
            the_event.process(self)

    def schedule(self, event):
        self.event_queue.put(event)


# A parent class for all random number generators
class RandomNumberGenerator:
    def next_random(self, handler: EventHandler):
        return 0


# A customer. Something who waits in queue to get served. Overide all methods for a special kind of customer.
class Customer:
    def __init__(self, name):
        self.name = name
        self.arrival_time = -1
        self.start_service_time = -1
        self.end_service_time = -1
        self.departure_time = -1
        self.my_server = None

        self.my_servers = []
        self.arrival_times = []
        self.start_service_times = []
        self.end_service_times = []

        self.was_served = False
        self.depart_on_service_done = True

    def will_balk(self, service_queue):
        return False

    def handle_arrival(self, handler: EventHandler):
        self.arrival_time = handler.time_now
        self.arrival_times.append(handler.time_now)

    def start_service(self, server, handler: EventHandler):
        self.my_server = server
        self.start_service_time = handler.time_now
        self.start_service_times.append(handler.time_now)
        return

    def end_service(self, handler: EventHandler):
        self.my_servers.append(self.my_server)
        self.my_server = None
        self.end_service_time = handler.time_now
        self.end_service_times.append(handler.time_now)
        self.was_served = True

        # If I'm supposed to depart at the end of service, then I should do that now. Otherwise,
        # this will not execute and I better o something else.
        if self.depart_on_service_done:
            handler.schedule(DepartureEvent(handler.time_now, self))

    def handle_departure(self, handler: EventHandler):
        self.my_server = None
        self.departure_time = handler.time_now
        # If we depart mid-service, fix up the times and servers.
        if len(self.start_service_times) != len(self.end_service_times):
            self.my_servers.append(self.my_server)
            self.my_server = None
            self.end_service_time = handler.time_now
            self.end_service_times.append(handler.time_now)

    def __str__(self):
        return "Customer[name=" + self.name + "]"


# A server who serves a customer.
class Server:

    def __init__(self):
        self.now_serving = None
        self.my_queue = None
        self.service_time_rng = None
        self.customers_served = []
        self.start_service_times = []
        self.end_service_times = []

    def __init__(self, service_time_rng: RandomNumberGenerator):
        self.now_serving = None
        self.my_queue = None
        self.service_time_rng = service_time_rng
        self.customers_served = []
        self.start_service_times = []
        self.end_service_times = []

    # Store the customer into the server's now_serving variable and generate the amount of time this
    # service will require.
    def start_service(self, customer, handler: EventHandler):
        self.now_serving = customer
        self.start_service_times.append(handler.time_now)
        end_service_time = handler.time_now + self.compute_service_time(customer,handler)

        # Schedule an event.
        handler.schedule(ServiceDoneEvent(end_service_time, customer, self.my_queue))

    def end_service(self, handler: EventHandler):
        self.customers_served.append(self.now_serving)
        self.end_service_times.append(handler.time_now)
        self.now_serving = None

    # You can override this to generate a custom service time computation.
    def compute_service_time(self, customer, handler):
        if self.service_time_rng is None:
            return 0
        else:
            return self.service_time_rng.next_random(handler)

    def compute_utilization(self, total_time):
        sum = 0
        num_services = len(self.customers_served)
        for i in range(0,num_services):
            dt = self.end_service_times[i] - self.start_service_times[i]
            sum += dt

        return sum / total_time


# A Generic Event.
class Event:
    def __init__(self, time_occurred, event_type):
        self.time_occurred = time_occurred
        self.type = event_type
        return

    def process(self, handler: EventHandler):
        return

    def __lt__(self, other):
        if isinstance(other, Event):
            return self.time_occurred < other.time_occurred
        else:
            # Default return true...in case you're forced into a queue with something weird
            return True

    def __str__(self):
        return "Event[type=" + type + ", time_occurred=" + str(self.time_occurred) + "]"


# An event involving a customer, that may or may not be a queue event.
class CustomerEvent(Event):
    def __init__(self, time_occurred, customer: Customer, event_type):
        Event.__init__(self, time_occurred, event_type)
        self.customer = customer
        return

    def process(self, handler: EventHandler):
        return


# A queue event. Something that happens to a queue.
class QueueEvent(Event):
    def __init__(self, time_occurred, customer, service_queue, event_type):
        CustomerEvent.__init__(self, time_occurred, customer, event_type)
        self.service_queue = service_queue
        return

    def process(self, handler: EventHandler):
        return


# An event that models the arrival of some customer to a queue.
class ArrivalEvent(QueueEvent):
    def __init__(self, time_occurred, customer, service_queue):
        QueueEvent.__init__(self, time_occurred, customer, service_queue, "Arrival")
        return

    def process(self, handler: EventHandler):
        self.service_queue.process_arrival(self.customer)

    # Handle arrival events after departure, abandon and service done events.
    def __lt__(self, other):
        if isinstance(other, Event):
            if self.time_occurred == other.time_occurred:
                if isinstance(other, DepartureEvent) or isinstance(other, ServiceDoneEvent) or isinstance(other,
                                                                                                          AbandonEvent):
                    return False
                else:
                    return QueueEvent.__lt__(self, other)
            else:
                return QueueEvent.__lt__(self, other)
        else:
            return QueueEvent.__lt__(self, other)


# An event that models the completion of service -- useful in queue networks, where departure doesn't necessarily happen
# immediately.
class ServiceDoneEvent(QueueEvent):
    def __init__(self, time_occurred, customer, service_queue):
        QueueEvent.__init__(self, time_occurred, customer, service_queue, "ServiceDone")
        return

    def process(self, handler: EventHandler):
        self.service_queue.process_service_done(self.customer)

    # Handle before arrivals
    def __lt__(self, other):
        if isinstance(other, Event):
            if self.time_occurred == other.time_occurred:
                if isinstance(other, ArrivalEvent):
                    return True
                else:
                    return QueueEvent.__lt__(self, other)
            else:
                return QueueEvent.__lt__(self, other)
        else:
            return QueueEvent.__lt__(self, other)


# An event that models the departure of someone from the **system**.
class DepartureEvent(CustomerEvent):
    def __init__(self, time_occurred, customer):
        CustomerEvent.__init__(self, time_occurred, customer, "Departure")
        return

    def process(self, handler: EventHandler):
        self.customer.handle_departure(handler)

    # Handle before arrivals
    def __lt__(self, other):
        if isinstance(other, Event):
            if self.time_occurred == other.time_occurred:
                if isinstance(other, ArrivalEvent):
                    return True
                else:
                    return QueueEvent.__lt__(self, other)
            else:
                return QueueEvent.__lt__(self, other)
        else:
            return QueueEvent.__lt__(self, other)


# An event that models the abandonment of someone from a queue.
class AbandonEvent(Event):
    def __init__(self, time_occurred, customer, service_queue):
        QueueEvent.__init__(self, time_occurred, customer, service_queue, "Abandon")
        return

    def process(self, handler: EventHandler):
        self.service_queue.process_abandon(self.customer, self.time_occurred)

    # Handle before arrivals
    def __lt__(self, other):
        if isinstance(other, Event):
            if self.time_occurred == other.time_occurred:
                if isinstance(other, ArrivalEvent):
                    return True
                else:
                    return QueueEvent.__lt__(self, other)
            else:
                return QueueEvent.__lt__(self, other)
        else:
            return QueueEvent.__lt__(self, other)


# A service queue (a resource in SimPy) this is something you wait in and ultimately get served in.
class ServiceQueue:
    def __init__(self, handler: EventHandler):
        self.queue = collections.deque()
        self.servers = []
        self.event_handler = handler

    def queue_length(self):
        return len(self.queue)

    def get_free_server(self):
        available_servers = []
        for server in self.servers:
            if server.now_serving is None:
                available_servers.append(server)

        if len(available_servers) == 0:
            return None
        else:
            return random.choice(available_servers)

    def add_server(self, server):
        server.my_queue = self
        self.servers.append(server)

    def call_next(self, server):
        if len(self.queue) > 0:
            next_customer = self.queue.pop()
            next_customer.start_service(server, self.event_handler)
            server.start_service(next_customer, self.event_handler)
        return

    def process_service_done(self, customer: Customer):
        # Write code to process a departure from server
        server = customer.my_server
        leaving_customer = server.now_serving
        leaving_customer.end_service(self.event_handler)
        server.end_service(self.event_handler)
        self.call_next(server)
        return

    def process_arrival(self, customer: Customer):
        # Write code to process arrival
        if customer.will_balk(self):
            customer.handle_departure()
            return

        customer.handle_arrival(self.event_handler)
        if self.queue_length() == 0:
            available_server = self.get_free_server()
            if available_server is None:
                self.queue.appendleft(customer)
            else:
                self.queue.appendleft(customer)
                self.call_next(available_server)
        else:
            self.queue.appendleft(customer)

        return

    # Allow customers to abandon if they're not in service.
    def process_abandon(self, customer: Customer):
        # Write code to process an abandonment
        if self.queue.count(customer) > 0:
            self.queue.remove(customer)
            customer.handle_departure(self.event_handler)
        return
    
class ExponentialRNG(RandomNumberGenerator):
    def __init__(self,mean_time_between_arrivals):
        self.mean_time_between_arrivals = mean_time_between_arrivals

    def next_random(self, handler: EventHandler):
        return random.expovariate(1.0/self.mean_time_between_arrivals)


**Part 1** As you did in Question 1, fill in the code needed to allow an <code>ExpoCustomer</code> to automatically generate the next arrival.

In [5]:
class ExpoCustomer(Customer):
    #This is a class variable. You can get at it by using ExpoCustomer.expo_customer_count. It's the *same* for every
    #ExpoCustomer.
    expo_customer_count = 0
    
    
    def __init__(self, name, arrival_time_rng: RandomNumberGenerator, service_queue: ServiceQueue):
        Customer.__init__(self, name)
        self.service_queue = service_queue
        self.arrival_time_rng = arrival_time_rng

    def schedule_next_arrival(self, handler: EventHandler):
        #PART 1 -- Fill in the code to allow this customer to schedule the next arrival time for the next customer.
        


    #Override the handle arrival event...now you can create the next vehicle in here.
    def handle_arrival(self, handler: EventHandler):
        Customer.handle_arrival(self, handler)
        self.schedule_next_arrival(handler)
        

**Part 2** As you did in Question 2, fill in the code for <code>compute_w_hat(vehicles)</code>. You may assume that each <code>Customer</code> has an <code>arrival_time</code> and a <code>departure_time</code>. So the total time a customer spends in the system is <code>customer.departure_time - customer.arrival_time</code>.

In [6]:
def get_all_customers(service_queue: ServiceQueue):
    all_customers = []
    for server in service_queue.servers:
        all_customers = all_customers + server.customers_served
    
    return all_customers
    
def compute_w_hat(customers):
    #PART 2 - Fill in the code here. 
    

**Part 3** Below is a function that simulates an $M/M/c$ queue for a value of $c$ you input and returns $\hat{w}$. This function has been wrapped in a loop to generate two lists of $\hat{w}$ values for $c = 1$ and $c = 2$. Write the code to use a t-test to tell if these two lists have equal means.

In [ ]:
def expo_simulation_c_servers(c):
    # This is the even handler.
    handler = EventHandler()

    # Create the service queue. Put the server at the end of it.
    service_queue = ServiceQueue(handler)
    
    # Create c servers.
    for i in range(0,c):
        server = Server(ExponentialRNG(15))
        service_queue.add_server(server)
    
    expo_rng = ExponentialRNG(20.0)
    #You have to start the world somehow.
    first_customer = ExpoCustomer("ExpoCustomer", expo_rng, service_queue)
    first_customer_time = expo_rng.next_random(handler)
    handler.schedule(ArrivalEvent(first_customer_time, first_customer, service_queue))

    handler.run(run_until_time=20000)
    
    all_customers = get_all_customers(service_queue)
    return compute_w_hat(all_customers)
    
X = []
Y = []
for i in range(0,10):
    print("Simulation run " + str(i+1))
    w = expo_simulation_c_servers(1)
    X.append(w)
    
    w = expo_simulation_c_servers(2)
    Y.append(w)
    
#PART 3 Write code to test and see if X and Y have the same mean. Either way, explain your results. 
#Explain also whether you assumed the samples had equal variance or not.



_Explain your results here by double clicking..._